# 自动微分

在梯度计算中，梯度是损失函数关于模型参数的导数。通常我们有两种直观的方法计算导数，但它们在深度学习面前都显得力不从心：

1. **数值微分**：

我们可以通过数值模拟的方法，给每个参数增加一个微小的扰动 $ϵ$ 来近似计算梯度：

$$
\frac{\partial J}{\partial w_i} \approx \frac{J(w_i + \epsilon) - J(w_i)}{\epsilon}
$$

但这种方法在深度学习中是不可接受的。假设我们的网络模型有 100 万个参数，为了得到全部梯度，我们需要进行 100 万次前向传播计算（运行 100 万次模型推理），这会造成灾难性的计算开销。

2. **符号微分**：

另一种方法，是像我们在第一部分里做的那样，手动推导出损失函数的数学解析式。

对于简单的线性回归，手动推导很容易。但是如果我们有 100 层网络，且包含各种更复杂的变换，手动推导出的导数公式可能就有数页。而一旦我们修改了网络结构（比如加个层），所有公式又要推倒重来，极其缺乏灵活性。

在深度学习的实践中，普遍采用的是第三种方法：

3. **自动微分**：

**自动微分**（Automatic Differentiation）的实现依据是微积分的**链式法则**（Chain Rule）；它的设计思路是：任何复杂的运算，在计算机底层都是由一系列基本运算（如加、减、乘、除、指数、对数）组合而成的，所以我们可以:

* 分解计算图：我们将复杂的公式拆解为一个计算图。每一个节点只负责一个简单的基本运算；
* 局部求导：对于每个基本运算（如 $p=w \cdot x + b$ ），我们可以预先定义好它的求导规则；
* 根据微积分的链式法则：

$$
\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}
$$

当我们从损失函数开始梯度计算时，模型并不需要知道所有导数的完整复杂公式，它只需要从损失值开始倒着走：
* 拿到后一层传来的上游梯度；
* 乘上当前层的局部导数；
* 结果继续传给父节点（前一层节点）。

通过这种方式，我们只需要执行一次前向传播（记录计算图），和一次梯度计算（应用链式法则），就能计算出所有参数的梯度。


In [1]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

计算图中的所有节点都是张量。

张量不仅是保存数据的地方，也是计算梯度的地方，是梯度计算链路传播的地方。为此，我们增加了三个属性：

* **梯度**（grad）：保存本张量的梯度
* **梯度计算函数**（gradient_fn）：计算局部导数
* **父节点**（parents）：构成梯度计算链路

同时，我们添加了一个函数：
* **反向函数**（backward）：调用梯度计算函数计算局部导数，并调用父节点中所有前一层节点进行梯度计算。这是一个递归调用过程，将触发所有上级节点，完成全部梯度计算。

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return len(self.data)

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础层

In [3]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def __repr__(self):
        return ''

### 基础损失函数

In [4]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

## 数据

### 特征、标签

In [5]:
feature = Tensor([28.1, 58.0])
label = Tensor([165])

## 模型

### 线性层

在线性层，我们要在**预测值张量**中添加线性回归的梯度计算函数，并将结果分别保存在**权重张量**和**偏置张量**的梯度（grad）中。

In [6]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad * x.data
            self.bias.grad += np.sum(p.grad, axis=0)

        p.gradient_fn = gradient_fn
        return p

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 损失函数（均方误差）

损失函数是梯度计算的起点。所以我们也需要在**损失值张量**中添加均方误差的梯度计算函数，并将结果保存在**预测值张量**的梯度（grad）中。

同时，要将预测值添加到父节点列表中，成为计算图的一部分。

In [7]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

## 验证

### 建模

In [8]:
layer = Linear(feature.size, 1)
print(layer)

Linear[weight(1, 2); bias(1,)]


### 推理

In [9]:
prediction = layer(feature)
print(f'prediction: {prediction}')

prediction: Tensor([43.05])


我们在开始模型推理的同时，开始构建动态计算图。

### 评估

In [10]:
loss_fn = MSELoss()
loss = loss_fn(prediction, label)
print(f'loss: {loss}')

loss: Tensor(14871.802500000002)


我们在评估模型的同时，也完成动态计算图的构建。

特征值是计算图的起点，损失值是计算图的终点。计算图的遍历节点目前只有一个，就是预测值。权重和偏置算是叶节点，不参与遍历。

```{figure} images/compu-graph.png
:align: center
:width: 300px
**图例：计算图结构**
```

* $x$：起点（特征值）
* $p$：遍历节点（预测值）
* $w, b$：叶节点（权重和偏置，不参与遍历）
* $l$：终点（损失值）

## 训练

### 梯度计算

现在，我们在梯度计算的时候，只需要调用**损失值张量**中的反向函数。它将：
* 计算损失值的梯度；
* 通过父节点列表，遍历计算图；
* 计算所有节点的梯度。

In [11]:
loss.backward()

### 反向传播

在反向传播的步骤，我们需要让每个参数张量减去它的的梯度值，就完成了一次迭代。

In [12]:
layer.weight.data -= layer.weight.grad
layer.bias.data -= layer.bias.grad

### 重新评估

In [13]:
prediction = layer(feature)
loss = loss_fn(prediction, label)
print(f'loss: {loss}')

loss: Tensor(1026548766283.6302)


我们升级了张量类，通过梯度计算函数实现了自动微分；通过父节点实现了动态计算图。至此，我们实现了梯度计算链路的自动化。